### Nama: Stanly Sukmajaya
### Batch: RMT-036
### Tujuan dari pembuatan notebook ini adalah untuk menganalisa data properties_rent_201501 yang diambil dari Google Big Query, yang kemudian hasil analisa tersebut di export ke dalam file .csv dan di buatkan API menggunakan FastAPI berdasarkan data yang dianalisa.

#### Melakukan koneksi dengan Google Big Query, menggunakan project_id pribadi

In [ ]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "pc-api-5133159006662443041-958" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


#### Membuat SQL query untuk memilih kolom `price` dari table `bigquery-public-data.properati_properties_ar.properties_rent_201501` yang di mana `operation` adalah **"rent"** dan `property_type` adalah **"apartment"**

In [47]:
df = client.query('''
SELECT price
FROM `bigquery-public-data.properati_properties_ar.properties_rent_201501`
WHERE operation = 'rent'AND property_type = 'apartment'
LIMIT 5000
''').to_dataframe()
df.head()

,price
0,2500.0
1,3500.0
2,2850.0
3,1850.0
4,1000.0


## Mencoba mendeteksi anomali di data tersebut

In [56]:
print("Mean:", df['price'].mean())
print("Median:", df['price'].median())
print("Mode:", df['price'].mode().values[0])
print(f"Skewness: {df['price'].skew()}")
print(f"Kurtosis: {df['price'].kurtosis()}")
print("")
print("Max:", df['price'].max())
print("Min:", df['price'].min())
print("Range:", df['price'].max() - df['price'].min())

Mean: 5166.355247285886
Median: 2700.0
Mode: 0.0
Skewness: 4.2401362864090695
Kurtosis: 25.47761685447876

Max: 100000.0
Min: 0.0
Range: 100000.0


### Setelah di cek, ternyata nilai skewness nya adalah 4.24, yang berarti data ini skewness nya sangat tinggi, berarti seharusnya ada banyak anomali

In [50]:
q1 = df['price'].quantile(0.25)
q3 = df['price'].quantile(0.75)
iqr = q3-q1

up_bound = q3 + 1.5 * iqr
low_bound = q1 - 1.5 * iqr

print(f'Upper Boundary: {up_bound:.2f}' )
print(f'Lower Boundary: {low_bound:.2f}')

outlier = df['price'][(df['price'] < low_bound) | (df['price'] > up_bound)]
print(len(outlier))
print(f'Percentage of outliers: {len(outlier)*100/len(df["price"]):.2f}%')

Upper Boundary: 13750.00
Lower Boundary: -8250.00
158
Percentage of outliers: 8.92%


### Karena nilai skewnessnya lebih dari 1, maka teknik yang digunakan untuk mendeteksi outliers adalah Tukey's rule. Berdasarkan analisa, presentase outliersnya adalah 8.92%

In [55]:
no_outliers = df[(df['price'] >= low_bound) & (df['price'] <= up_bound)]
print("Mean:", no_outliers['price'].mean())
print("Median:", no_outliers['price'].median())
print("Mode:", no_outliers['price'].mode().values[0])
print(f"Skewness: {no_outliers['price'].skew()}")
print(f"Kurtosis: {no_outliers['price'].kurtosis()}")
print("Max:", no_outliers['price'].max())
print("Min:", no_outliers['price'].min())
print("Range:", no_outliers['price'].max() - no_outliers['price'].min())
no_outliers.to_csv("properties_rent.csv", index=False)

Mean: 2875.078
Median: 2500.0
Mode: 0.0
Skewness: 1.3384880750664294
Kurtosis: 1.3918315638550864
Max: 13500.0
Min: 0.0
Range: 13500.0


### Setelah data nya dibersihkan data anomali nya (data yang anomali dibuang) terdapat perubahan yang cukup drastis di mean, yaitu menjadi 2875.078 dari 5166.355. Median berubah dari 2700 menjadi 2500, dan modus tetap sama, yaitu 0. Terjadi perubahan yang sangat drastis pada skewness dan kurtosis, yaitu skewness berubah menjadi 1.34 dari 4.24, dan kurtosis berubah menjadi 1.39 dari 25.48. Sebagai tambahan, jarak antara nilai maksimum dan minimum berubah drastis menjadi 13500, dari 100000. Bisa disimpulkan bahwa meskipun nilai skew dan nilai persebaran data nya masih cukup tinggi, namun sudah jauh lebih baik dari data awal.

### 1. Berapa rata-rata, median, dan modus dari data tersebut sebelum dihilangkan outliernya? Bagaimana kecerendungan pemusatan datanya? jelaskan jawabanmu!

In [54]:
print("Mean:", df['price'].mean())
print("Median:", df['price'].median())
print("Mode:", df['price'].mode().values[0])
print("")
print("Max:", df['price'].max())
print("Min:", df['price'].min())
print("Range:", df['price'].max() - df['price'].min())

Mean: 5166.355247285886
Median: 2700.0
Mode: 0.0

Max: 100000.0
Min: 0.0
Range: 100000.0


### Dari data yang di dapat di atas, bisa dilihat bahwa nilai mean, median dan mode nya sangat jauh antara satu sama lain, yang bisa diartikan bahwa ada perbedaan jauh antar 1 data dengan data yang lainnya (data nya tersebar). Karena nilai mean dan median nya sangat jauh, maka untuk nilai central tendency, lebih baik menggunakan nilai median, karena tidak terlalu terpengaruh oleh outlier. Median juga lebih baik digunakan jika data nya memiliki nilai skew yang tinggi. Sebagai tambahan, jarak antara nilai minimum dan maksimumnya adalah 100000, yang bisa di bilang bahwa itu cukup jauh, sehingga memperkuat argumen bahwa dataset ini sangat tersebar.

### 2. Sebelum melakukan extreme value analysis, kamu harus melakukan pengecekan skewness dari distribusi datanya. Apakah datanya skew atau normal? jelaskan jawabanmu!

In [53]:
print(f"Skewness: {df['price'].skew()}")
print(f"Kurtosis: {df['price'].kurtosis()}")

Skewness: 4.2401362864090695
Kurtosis: 25.47761685447876


### Berdasarkan data skewness di atas, bisa dilihat bahwa nilai skewness nya sangat tinggi, yaitu 4.24. Secara standard, nilai skewness yang dibawah -1 atau di atas 1 mengartikan bahwa nilai skewness nya sangat tinggi. Jika nilainya 4.24, bisa disimpulkan bahwa data tersebut adalah highly skewed, dan untuk mengambil nilai central tendency, lebih baik menggunakan median, karena tidak terlalu terpengaruh oleh outlier dan lebih baik digunakan dengan data yang nilai skew nya tinggi.

### 3. Ada dua teknik untuk melakukan extreme value analysis, teknik yang mana yang kamu pakai? berikan alasanmu berdasarkan data!

#### Saya menggunakan teknik Tukey's rule, karena Tukey's rule lebih baik digunakan untuk data yang skewed, dan nilai skew nya kurang atau lebih dari 0.5.